# Roadmap

1. Get data
2. Create train-test split
3. Clean training set (write functions): missing values, text, categorical attributes, scaling
4. Select models and scoring metrics, then train
5. Compare them: clean test set, make predictions, score
6. Fine-tune models

## 1. Load data

In [1]:
from functions import load_data
data_df = load_data('data/time_series.xlsx')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 292 entries, 2012-04-08 to 2017-11-05
Columns: 1833 entries, 012 to TRUHONE
dtypes: int64(1833)
memory usage: 4.1 MB


## 2. Train-Test Split

In [2]:
# splitting into training and testing sets using 66-34
from functions import train_test_split

train_df, test_df = train_test_split(data_df, 0.66)
train_df.head()

Observations: 292
Training Observations: 192
Testing Observations: 100


,012,017,03008944ST-1,03008944ST-3,0300ST1550-1,0300ST15X9-1,0300ST15X9-2,0300ST15X9-3,0300ST1605-1,0300ST1605-2,...,9920-2,9920-3,9920-4,9920-5,9920-6,9920-7,9997-25,HW220D15,HW240DIA,TRUHONE
EntDate,,,,,,,,,,,,,,,,,,,,,
2012-04-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-04-15,0,0,0,0,0,0,0,0,0,0,...,20,19,7,166,98,0,0,0,0,0
2012-04-22,0,0,0,0,0,0,0,0,0,0,...,41,15,10,207,87,0,0,0,0,0
2012-04-29,0,0,0,0,0,0,0,0,0,0,...,38,44,13,101,21,10,0,0,0,0
2012-05-06,0,0,0,0,0,0,0,0,0,0,...,29,47,21,75,43,0,0,0,0,0


## Train-Test Split -- alternative

In [ ]:
# now we will make three datasets using 33-33-34
X = data_df.values
train_size1 = int(len(X) * 0.33)
train_size2 = int(len(X) * 0.66)
pretrain, train, test = X[0:train_size1], X[train_size1:train_size2], X[train_size2:len(X)]
print('Observations: %d' % (len(X)))
print('Initializing Observations: %d' % (len(pretrain)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

In [ ]:
# made train_df and test_df (the latter to be used later)
pretrain_df = data_df[0:train_size1]
train_df = data_df[train_size1:train_size2]
test_df = data_df[train_size2:292]
train_df.head()

## Select single item for forecasting

In [ ]:
# pick one item
test = train_df['9920-2']
test.head()

In [ ]:
# plot one item
from functions import plot_train_test
plot_train_test(train_df, test_df, '9920-2')

## 3. Clean Training Set

In [ ]:
# make a copy first

In [ ]:
# make transformations pipeline (first applied to train, then apply to test)
# luckily, our data has no null values, no categorical/text values
# however, to get the initial messy excel sheet into timeseries format was not easy
# describe

## 4. Select Models and Scoring Metrics

We are comparing different forecasting models on the same data, so we can use scale-dependent errors because our single dataset only has one scale. We will use two metrics: <br>
* MAD (also called MAE): $\frac{|A_t-F_t|}{n}$ <br>
* RMSE (root mean squared error): $\sqrt{\frac{(A_t-F_t)^2}{n}}$ <br>

For baseline, we will use:
* naive forecast ("only yesterday matters")

Then we will try two simple forecasts:
* cumulative ("history matters")
* moving average ("I select how much matters")

We will also try:
* ARIMA
* exponential smoothing

In [3]:
# make a df to store all our predictions
from functions import make_copy_df
y_hat = make_copy_df(test_df, '9920-2')

### Naive Forecast (Baseline)

In [ ]:
# FIX LATER
import numpy as np

dd= np.asarray(train_df['9920-2'])
y_hat['naive'] = dd[len(dd)-1] # this line of code is for one-time forecast

In [ ]:
# plot
from functions import plot_time_series
plot_time_series(train_df, test_df, '9920-2', y_hat, 'naive', 'Naive Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'naive')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df, '9920-2', y_hat, 'naive')

### Cumulative

In [ ]:
# FIX LATER
y_hat['cumulative'] = train_df['9920-2'].mean()
plot_time_series(train_df, test_df, '9920-2', y_hat, 'cumulative', 'Cumulative Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'cumulative')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'cumulative')

### Moving Average

In [ ]:
# FIX LATER
# since forecasts are for an entire year, we are using 52 weeks
y_hat['moving_avg'] = train_df['9920-2'].rolling(156).mean().iloc[-1]
plot_time_series(train_df, test_df, '9920-2', y_hat, 'moving_avg', 'Moving Average Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'moving_avg')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'moving_avg')

### S/ARIMA

In [ ]:
# gridsearch with pyramid
from pyramid.arima import ARIMA

fit = ARIMA(order=(1, 1, 0), seasonal_order=(1, 0, 0, 12)).fit(y=train_df['9920-2'])

from pyramid.arima import auto_arima

stepwise_fit = auto_arima(train_df['9920-2'], start_p=0, start_q=0, max_p=6, max_q=6, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
# fit with SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX

fit1 = SARIMAX(train_df['9920-2'], order=(0, 1, 1), seasonal_order=(0,1,1,12), freq='W').fit()
y_hat['SARIMA'] = fit1.predict(start="2015-12-13", end="2017-11-05", dynamic=True, typ='levels')

# plot
plot_time_series(train_df, test_df, '9920-2', y_hat, 'SARIMA', 'SARIMA Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'SARIMA')

### Exponential Smoothing

In [ ]:
# MAKE FUNCTION
# let's decompose it first to see trend, seasonality
from statsmodels.tsa.seasonal import seasonal_decompose
series = train_df['9920-2']
result_a = seasonal_decompose(series, model='additive')
result_a.plot()
plt.show()

In [ ]:
# MAKE FUNCTION
# let's plot it a bit bigger
fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(16,10))
result_a.trend.plot(ax=ax1, label='Trend')
result_a.seasonal.plot(ax=ax2)
result_a.resid.plot(ax=ax3)
plt.show()

In [ ]:
# Double Exponential Smoothing with Statsmodels
# no trend, just seasonality (multiplicative), no damping
from statsmodels.tsa.api import ExponentialSmoothing

fit1 = ExponentialSmoothing(np.asarray(train_df['9920-2']), seasonal_periods=10, trend=None, seasonal='additive').fit(smoothing_level=0.51, smoothing_seasonal=0.1)
y_hat['DES'] = fit1.forecast(len(test_df))

# plot
plot_time_series(train_df, test_df, '9920-2', y_hat, 'DES', 'Double ES Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'DES')

In [ ]:
# here we have trend and seasonality, so we will use Holt-Winters with Statsmodels
# additive trend and seasonality, no trend damping, seasonal periods=10
from statsmodels.tsa.api import ExponentialSmoothing

fit1 = ExponentialSmoothing(np.asarray(train_df['9920-2']), seasonal_periods=10, trend='add', seasonal='add').fit(smoothing_level=0.51, smoothing_slope=0.015,smoothing_seasonal=0.1)
y_hat['Holt_Winter'] = fit1.forecast(len(test_df))

plot_time_series(train_df, test_df, '9920-2', y_hat, 'Holt_Winter', 'Holt_Winter Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'Holt_Winter')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'Holt_Winter')

### Exponential Smoothing with Homebrewed Functions

In [ ]:
from exponential_smoothing import initial_trend, initial_seasonal_components, triple_exponential_smoothing

In [ ]:
predictions = triple_exponential_smoothing(train_df['9920-2'], slen=10, alpha=0.51, beta=0.015, gamma=0.1, n_preds=100)
y_hat['HW_new'] = predictions[-100:]

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'HW_new')

### Exponential Smoothing with Homebrewed Class

In [4]:
from HoltWinters_class import HoltWinters

In [5]:
model = HoltWinters(alpha=0.51, beta=0.015, gamma=0.1)
model.fit(train_df.index, train_df['9920-2'])
preds = model.predict(X=train_df['9920-2'], slen=10, n_preds=100)
y_hat['HW_Class']=preds[192:292]

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'HW_Class')

In [6]:
# calculate RMSE
model.score(train_df['9920-2'], test_df['9920-2'], slen=10, n_preds=100)

97.34503609614828

### Prophet

In [ ]:
# imports 
from fbprophet import Prophet

In [ ]:
# make a copy of the dataframe for Prophet transformations
from functions import make_copy_df
prophet_df = make_copy_df(train_df, '9920-2')

In [ ]:
# rename variables (prophet requires the variable names in the time series to be
# y for target and ds for Datetime)
from functions import rename_columns
rename_columns(prophet_df, '9920-2')
prophet_df.head()

In [ ]:
# FIX LATER
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(growth='linear', interval_width=0.95, weekly_seasonality=True)
my_model.fit(training)

In [ ]:
# FIX LATER
future_dates = my_model.make_future_dataframe(periods=100, freq='W')
forecast = my_model.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# FIX LATER
forecast.head()
forecast = forecast.set_index('ds')
forecast_slice=forecast[192:292]
forecast_df = forecast_slice["yhat"]
forecast_df.head()

In [ ]:
# plot
plot_time_series(train_df, test_df, '9920-2', forecast_slice, 'yhat', 'Prophet Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', forecast_slice, 'yhat')

## Resources

Great basic blogs on time series and forecasting methods: https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c

On Prophet:
https://www.analyticsvidhya.com/blog/2018/05/generate-accurate-forecasts-facebook-prophet-python-r/
https://research.fb.com/prophet-forecasting-at-scale/
(Plus see 